In [3]:
import numpy as np
import pandas as pd
import re,math
import concurrent.futures
import multiprocessing
import threading
import logging
import time

# 设置日志级别
logging.basicConfig(level=logging.DEBUG)

# 设置函数运行计时用的修饰器
def timeit(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'{func.__name__} took {end - start:.6f} seconds')
        return result
    return wrapper

# 高相似多少
coum=200
# 加载保存的数组
lin = np.load("lin.npy")
# lin = np.linspace(0,3,401)
# lin = np.linspace(0.01,1,100)


data1 = pd.read_excel(r'pcbi.1006752.s002.xlsx', sheet_name="Sheet1",index_col='Drug').drop('SMILES',axis=1)
data2 = pd.read_excel(r'583drugs39cell Zscore行标准化 synergy.xlsx')
cells_name = pd.read_excel(r'583drugs39cell Zscore行标准化 synergy.xlsx', header=0).columns[2:] # 保证cells的名字从第一行的第三个开始
drug_names = list(data1.index)

# 合并drugA和drugB为一列作为索引
# 顺序
temp1 = [] # 分子指纹合并值 【【】，【】，【】，】
index_1 = [] # 对应的名字列表 【【】，【】，【】，】
# 逆序
temp2 = []
index_2 = []
for index,row in data2.iterrows():
    ct = list((np.array(data1.loc[row["DrugA"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugB"],:]))))
    temp1.append(ct)
    index_1.append((row["DrugA"],row["DrugB"]))
    ct = list((np.array(data1.loc[row["DrugB"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugA"],:]))))
    temp2.append(ct)
    index_2.append((row["DrugB"],row["DrugA"]))

/root/miniconda3/envs/jupyter-env/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [4]:
# 两组数据的tanimoto计算
def tanimoto(p,q):
    tep1=0
    tep2=0
    lenthp=int(len(p))
    for i in range(lenthp):
        a=p[i]
        b=q[i]
        if (a==1)|(b==1): # 并
            tep1=tep1+1
        if (a==1)&(b==1): # 交
            tep2=tep2+1
    c=round((tep2 / tep1),4)      #取值4位数
    return c

def compute_tanimoto(i, j, i_n, j_n, temp1, temp2, index_1, index_2, idx):
    tmp=[]
    a = np.array(i) #取出一分子指纹 转换列表为向量
    b = np.array(j)
    for k,m,k_n,m_n in zip(temp1,temp2,index_1,index_2): # 遍历每一个药物对的分子指纹
        c = np.array(k)
        d = np.array(m)
        tp1 = tanimoto(a,c)
        tp2 = tanimoto(a,d)
        tp3 = tanimoto(b,c)
        tp4 = tanimoto(b,d)
        tps = [tp1, tp2, tp3, tp4]
        index_list = [(i_n,k_n,tp1),(i_n,m_n,tp2),(j_n,k_n,tp3),(j_n,m_n,tp4)]
        index_location = tps.index(max(tps))
        tmp.append(max(tps))
        Tanimoto_index_dist[f"{i_n},{k_n}"] = index_list[index_location]
    return idx, tmp

temp = [None] * len(temp1)
Tanimoto_index_dist = {}
with concurrent.futures.ThreadPoolExecutor() as executor:
    # 提交任务时使用 enumerate ，并将索引值作为参数传递给任务函数
    futures = [executor.submit(compute_tanimoto, i, j, i_n, j_n, temp1, temp2, index_1, index_2, idx)
               for idx, (i,j,i_n,j_n) in enumerate(zip(temp1,temp2,index_1,index_2))]
    for future in concurrent.futures.as_completed(futures):
        idx, result = future.result()
        temp[idx] = result
Tanimoto = pd.DataFrame(temp, columns=index_1, index=index_1)

In [5]:
# 获取columns数组 - 处理后的
ttpp = []
for i in list(Tanimoto.columns):
    ttpp.append(str(i)[1:-1]) # 去除首尾括号的药物对名字
data2 = pd.DataFrame(data2.iloc[:,2:].T.values, columns=list(ttpp)) # 药物对协同得分
columns = data2.columns # 修改后的药物对名字

In [6]:
# 删去对应行后的字典
dict_lines = {} # {键1：值1} 谷本系数字典
count = 0
ct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,ct]) # 每一列谷本系数 [:,1] 第一列的每一行 实际就是第一列
    y.pop(count)
    count += 1
    dict_lines[columns[ct]] = y # 谷本系数药物对的名字做键1，y是值1
    ct += 1

In [7]:
# 分离x y数据构造字典
dict_columns_split = {} # 高相似字典
# 计算x数据
x = data2.drop(columns=columns)  # 其他所有的列
for i in columns:
    # b 要预测协同得分那一列
    y = data2.loc[:, i]
    # A 权重
    x_i = x.copy()
    x_i['y'] = y

    # 高相似的    取高相似的药物对名字max1和对应的值max5
    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_lines[i], index = tp)
    df.sort_values(by=0, inplace=True, ascending=True)
    max_l = df.index[-coum:]
    max_5x = pd.DataFrame([data2.loc[:, j] for j in max_l]).T

    # 存储 对应数据
    count = 0 # 第几行 int
    for j in list(y.index): # 35个索引 0~34 数字（字符串-str） 【0，1，2，3.。。。】
        dict_columns_split[f'{i},{j}']={} # 空字典 键-空字典
        dict_columns_split[f'{i},{j}']['y_line']=y[count]
        dict_columns_split[f'{i},{j}']['max_x_line']=max_5x.iloc[count,:].values
        count += 1

In [8]:
TTT = [] # 值列表
cct = 0
NNN = []
for i in Tanimoto.index: 
    y = list(Tanimoto.iloc[:,cct])
    y.pop(cct) 
    tp_columns = list(columns.copy())
    tp_columns.pop(cct)
    df = pd.DataFrame(y, index=tp_columns)
    df.sort_values(by=0 , inplace=True, ascending=True) 
    max_l = df.index[-coum:]
    NNN.extend(max_l)
    # 计算max_5y
    max_5y = []
    for j in max_l:
        max_5y.append(Tanimoto.iloc[list(columns).index(j), cct])
    # 将max_5y加入TTT
    TTT += max_5y
    # 修改cct变量的值
    cct += 1

In [9]:
def alpha_pcc(alp, t_coum, a, linename, dict_columns_split, TTT, coum, data2, NNN):
    alpha = []
    result_pcc = []
    pcc_split = []
    pre = []
    ori_dict = []
    pre_dict = []
    for elem in range(39):
        pp = []
        for j in TTT[(coum * a - t_coum):(coum * a)]: 
            pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))#改函数
        pre.append(np.sum(pp / np.sum(pp) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]))
    pre_dict.append(pre)
    ori_dict.append(data2[linename])
    pcc_split.append(np.corrcoef(pre, data2[linename])[0, 1])
    result_pcc.append(np.max(pcc_split))
    pcc=result_pcc[0]
    pre = pre_dict[pcc_split.index(np.max(pcc_split))]
    ori = ori_dict[pcc_split.index(np.max(pcc_split))]
    temp_NNN = NNN[(coum * a - t_coum):(coum * a)]
    temp_TTT = TTT[(coum * a - t_coum):(coum * a)]
    del result_pcc, pcc_split, pp
    return pcc, temp_NNN, temp_TTT, pre, ori

def alp_mse_rmse(alp, t_coum, a, linename, dict_columns_split, TTT, coum):
    r1 = []
    ct = 0
    for elem in range(39):
        pp = []
        for j in TTT[(coum * a - t_coum):(coum * a)]:
            pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))  # 改函数
        r1.append(np.sum((pp/np.sum(pp)) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]) -
                        dict_columns_split[f"{linename},{elem}"]['y_line'])
    mse=np.mean(np.square(r1))
    rmse=math.sqrt(mse)
    return mse, rmse

In [10]:
# 单进程 分函数
def calculate_parameters(alp, t_coum, coum, TTT, data2, dict_columns_split, lin, NNN):
    a = 1
    Name_data = []
    for linename in columns:
        # 求PCC
        pcc, temp_NNN, temp_TTT, pre, ori = alpha_pcc(alp, t_coum, a, linename, dict_columns_split, TTT, coum, data2, NNN)
        # 求MSE
        mse, rmse = alp_mse_rmse(alp, t_coum, a, linename, dict_columns_split, TTT, coum)
        Name_data.append([linename, t_coum, alp, mse, pcc, rmse, temp_NNN, temp_TTT, pre, ori])
        a = a+1
    pcc_mean = pd.DataFrame(Name_data, columns=['drug', 'coum', "alp", "mse", "pcc", "rmse", "temp_NNN", "temp_TTT", "pre", "ori"])["pcc"].mean()
    print(pcc_mean, alp)
    return [pcc_mean, alp, Name_data]

In [ ]:
# 多进程
@timeit
def run_parallel(columns, lin, NNN):
    with multiprocessing.Pool() as pool:
        tp = []
        for alp in lin:
            for t_coum in range(2, coum+1, 1):    #邻居
                tp.append((alp, t_coum, coum, TTT, data2, dict_columns_split, lin, NNN,))
        results = pool.starmap(calculate_parameters, tp,)
    return results

PCC_mean = 0
res_pcc_list = []
results = run_parallel(columns,lin,NNN)

0.31403596652673277 1.0
0.34241853667438493 1.0
0.3742804329280965 1.0
0.5768039829008392 1.015
0.4065966624993245 1.0
0.4269636917850709 1.0
0.44379223225657577 1.0
0.5789169866058282 1.015
0.6112139375572742 1.03
0.563897737287989 1.07
0.4552562183636186 1.0
0.48970167933828024 1.005
0.46626341186601555 1.0
0.5639677086951811 0.58041122119317261.045 
1.015
0.5672256956743905 1.07
0.6116004848761362 1.085
0.5460236065707919 1.125
0.4811927748137632 1.0
0.6104449412785024 1.03
0.44137367741980504 1.02
0.5695546125422516 1.07
0.49092424546022206 1.0
0.4996436316927354 1.06
0.58316629996507 1.015
0.5494204967225623 1.125
0.5758382433986947 1.1
0.608225494379816 1.1400000000000001
0.49968181589422644 1.0
0.5144318516416418 1.18
0.6108544386765757 1.085
0.5728311586097959 1.07
0.4018187958438511 1.035
0.5536862907262426 1.125
0.5061759650832836 1.0
0.5853854944460302 1.015
0.44840201489222964 1.075
0.5235487186113266 1.18
0.5108167032446795 1.115
0.6100060258966064 1.03
0.5870410879942719 

In [ ]:
sorted_list = sorted(results, key=lambda x: x[0])
max_res = sorted_list[-1][2]
print("MAX PCC: ", sorted_list[-1][0])
print("ALP:", sorted_list[-1][1])

In [ ]:
results = []
for i in max_res:
    results.append([i[0],i[1],i[2],i[3],i[4],i[5]])

In [ ]:
# 将结果保存到 CSV 文件中
pd.DataFrame(results).to_csv('pcc_cpu_test_multi_unite_alp.csv', header=None, index=False)
rebuild_list = []
for i in max_res:
    for j,k in zip(i[6],i[7]):
        rebuild_list.append([j,i[0],k,i[1],i[2],i[4],i[5]])
pd.DataFrame(rebuild_list,columns=["target_neighbor","target","similarity","neighbors","alp","pcc","rmse"]).to_csv("pcc_cpu_test_multi_target_unite_alp.csv")
temp_list = []
for i in max_res:
    for j,k,m in zip(i[8],i[9],cells_name):
        temp_list.append([i[0],m,j,k])
pd.DataFrame(temp_list,columns=["drugs","cells_line","pridict","origin"]).sort_values("cells_line").to_csv("pcc_cpu_test_multi_cells_line_unite_alp.csv")